# E2E Approach -- Scenario I

In this notebook, we report the code related to the *E2E* appraoch in our paper [[1]](#ourpaper). We consider the delay caused by the feedback loop used by the mobile user equipment to send to the base station the channel state information (CSI). Due to this feedback delay, the CSI available at the base station becomes outdated. In our paper, we present an E2E appraoch, which takes in input the channel history available at the base station and maps it to the probability of an error event for all the modulation and coding schemes (MCSs) the base station can select. The error event corresponds to an unsuccessfully decoded frame at the receiver. If the error event probabilities are known for all the MCSs, then it is straightforward to select the MCS that maximizes the spectral efficiency.

Under Scenario I, we train a different neural network for each combination of delay, doppler, and signal-to-noise-ratio.

It must be noted that the training datasets listed below in the code are currently not available in the repository due to space limitations. The **training datasets can be found at**: https://kth.box.com/s/tcd7y7rg3yau75kctw3regmyns8kfkr6 in the folder *Trained_folder_ScenarioI*. At any rate, in the repository, the reader can also find the codes in *radio_data* folder which can be run to generate the datasets.

**Note**: the training might take some hours, depending on the available computational resources, the dimension of the training set, the dimension of the network, and the number of epochs. 

<a id='ourpaper'></a> [1] "Wireless link adaptation - a hybrid data-driven and model-based approach", Lissy Pellaco, Vidit Saxena, Mats Bengtsson, Joakim Jaldén. Submitted to WCNC 2020.

## Import libraries and utility functions

In [1]:
import numpy as np
import utilities as utils
from keras.optimizers import Adam
from keras.backend.tensorflow_backend import set_session
from keras.backend import clear_session
import tensorflow as tf
import time

Using TensorFlow backend.


In [ ]:
# Number of MCSs
NROF_MCS=29
# Number of subcarriers in the OFDM
NROF_SUBCARRIERS=72
# Flag used to indicate if the channel is noisy
CHANNEL_EST_NOISE = True

# Number of past CSI stored at the base station and used as input to the neural network
E2E_MEMORY = 10
BATCH_SIZE = 32
NROF_EPOCS = 20 
TRAINING_FRACTION = 1

# Flag to indicate if the trained models should be saved
save_model = False

## Load the Dataset

The channel dataset is a dict with the following keys :  
 - 'channel'
     - Complex channel coefficients 
     - Numpy array [ NROF_FRAMES x NROF_SUBCARRIERS x NROF_SNRS]
 - 'block_success'
      - Binary success events (ACKs)
      - Numpy array [ NROF_FRAMES x NROF_MCS x NROF_SNRS]
 - 'snrs_db '      
     - Evaluated average SNR values
     - Numpy array [ NROF_SNRS ]
 - 'block_sizes'
     - Evaluated transport block sizes
     - Numpy array [ NROF_MCS ]
     
The name of the dataset, e.g., ITU_VEHICULAR_B_1000_210_111_72_5dB, is to be interpreted in this way: 
 - channel model
 - number of channel realizations per batch
 - number of batches of the dataset
 - doppler in Hz (cast to integer)
 - number of subcarriers
 - snr
 
The **training datasets can be found at**: https://kth.box.com/s/tcd7y7rg3yau75kctw3regmyns8kfkr6 in the folder *Datasets*

In [ ]:
# The files stored in the file_set ARE NOT in the repository due to space limitations.
# The training datasets can be found at: https://kth.box.com/s/tcd7y7rg3yau75kctw3regmyns8kfkr6 in the folder *Datasets*
# The reader has also access to the "radio_data/Generate_Data.ipynb" which we used to generate the training datasets.

file_set = [
            'Datasets/ITU_VEHICULAR_B_1000_210_16.67_72_5dB.npy',
          'Datasets/ITU_VEHICULAR_B_1000_210_16.67_72_15dB.npy',
          'Datasets/ITU_VEHICULAR_B_1000_500_16.67_72_25dB.npy',
        ]
# Doppler frequency related to the datasets in "file_set"
dopplers_set = (20.0 / 3.0) * np.array([16.67,16.67,16.67])

# Snrs related to the datasets in "file_set" 
snrs_set = [5,15,25]

# Number of batches related to the datasets in "file_set"
num_batches_per_dataset = [210,210,500]

# Maximum feedback delay that we consider
maximum_delay = 9

## Build the Neural Network model, define the optimizer, and the cost function

In [ ]:

def create_ann_model():
    from keras.models import Sequential
    from keras.layers import Dense, Dropout

    model = Sequential()
    model.add( Dense( 1024,
                      input_dim = NROF_SUBCARRIERS * E2E_MEMORY * 2, 
                      kernel_initializer='normal', 
                      activation='relu' ) )

    model.add( Dense( 512, 
                      kernel_initializer = 'normal', 
                      activation='relu' ) )

    model.add( Dense( 1024,
                      kernel_initializer = 'normal', 
                      activation='relu' ) )
    
    model.add( Dense( NROF_MCS, 
                      kernel_initializer='normal', 
                      activation='sigmoid' ) )

    # Compile model
    adam = Adam(lr = 0.0001, beta_1 = 0.9, beta_2 = 0.999, amsgrad = False)
    model.compile(loss = 'binary_crossentropy', optimizer = adam, metrics = ['accuracy'])  # for binary classification

    
    return model



## Train the Neural Network

In [ ]:
start_time = time.time()

for DELAY in range(0,maximum_delay + 1):
    
    for i in range(0,len(snrs_set)):  
        
        DOPPLER=dopplers_set[i]
        FILE=file_set[i]
        SELECTED_SNR=snrs_set[i]
        DATASET = np.load( FILEfile, allow_pickle = True )[()]
        
        config = tf.ConfigProto()
        config.gpu_options.allow_growth = True  

        sess = tf.Session( config = config )
        set_session(sess) 
        

        model = create_ann_model( )

        for j in range(0,num_batches_per_dataset[i]):    

            channel_coeff  = []
            block_success  = []
        
            DATASET_channel=(DATASET['channel'][:,:,j])  
            DATASET_block_success=(DATASET['block_success'][:,:,j]) 

            nrof_train_samples = int( TRAINING_FRACTION * DATASET_channel.shape[0] )
            coeff = utils.calculate_channel_coefficients_scaled_fixed_snr( DATASET_channel[ :nrof_train_samples, : ],
                                                                 SELECTED_SNR,
                                                                 channel_estimation_noise = CHANNEL_EST_NOISE )

            channel_coeff.append( coeff )
            block_success.append( DATASET_block_success[ :nrof_train_samples, : ] )

            channel_coeff = np.vstack( channel_coeff )
            block_success = np.vstack( block_success )


            BLOCK_SIZES = DATASET[ 'block_sizes' ] 


            NROF_FRAMES,_= channel_coeff.shape


            channel_coeff_concat = np.concatenate( ( np.real( channel_coeff ), np.imag( channel_coeff ) ), axis = 1 )


            if DELAY > 0:
                train_input  = ( channel_coeff_concat[ :-DELAY, :] )
            else:
                train_input  = ( channel_coeff_concat[:,:] )

            train_target = ( block_success [ DELAY :, :] )

            train_input, train_target = utils.shuffle_data( train_input, train_target )



            history = model.fit( utils.stack_features( train_input, ANN_MEMORY ), 
                                         train_target, 
                                         batch_size = BATCH_SIZE, 
                                         epochs     = NROF_EPOCS, 
                                         validation_split = 0.1, 
                                         verbose    = 0 ) # the "verbose parameter" can be changed to display more about the training progess of each epoch


        
        file_to_save = 'Trained_models_ScenarioI/ANN_E2E_MCS_PRED_DELAY_%d_DP_%d_SNR_%d.h5'%(DELAY,DOPPLER,SELECTED_SNR)
        if save_model == True:      
            model.save( file_to_save )

        print("--- %s seconds ---" % (time.time() - start_time))
        clear_session( )
